In [1]:
import gzip
from collections import defaultdict
import random
from sklearn.svm import SVC  
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import classification_report, precision_score
from multiprocessing import Process, Queue

from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import Reader
from surprise import KNNWithMeans
from surprise import SVD
from surprise import SVDpp
import pandas as pd
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate

def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

C:\Users\nabic\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [5]:
#Alt solution for purchase prediction
userID = []
itemID = []
categoryID = []
ratingAll = []
purchased = []
for l in readGz("train.json.gz"):
    user,business,category,rating = l['reviewerID'],l['itemID'], l['categoryID'], l['rating']
    userID.append(user)
    itemID.append(business)
    categoryID.append(category)
    ratingAll.append(rating)
    purchased.append(1)
# for user in negatives:
#     for item in negatives[user]:
#         userID.append(user)
#         itemID.append(item)
#         purchased.append(0)
#         ratingAll.append(0.0)
allDataDict = {'reviewerID':userID,
              'itemID':itemID,
              'purchase':purchased}
df = pd.DataFrame(allDataDict)
reader = Reader(rating_scale=(0,1))
data = Dataset.load_from_df(df[['reviewerID','itemID','purchase']],reader)
trainset, testset = train_test_split(data, test_size=.15,random_state=7)
algo = SVDpp(n_factors=140, n_epochs=100, lr_all=0.003, reg_all=0.15,verbose=True)
# algo = KNNWithMeans(verbose=True,sim_options={'name':'cosine', 'user_based':False})
algo.fit(trainset)
test_pred = algo.test(testset)
accuracy.rmse(test_pred,verbose=True)
# param_grid = {'n_factors': [110, 120, 140], 'n_epochs': [100], 'lr_all': [0.003, 0.005],
#               'reg_all': [0.08, 0.1, 0.15]}
# gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
# gs.fit(data)
# algo = gs.best_estimator['rmse']
# print(gs.best_score['rmse'])
# print(gs.best_params['rmse'])
# cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 processing epoch 4
 processing epoch 5
 processing epoch 6
 processing epoch 7
 processing epoch 8
 processing epoch 9
 processing epoch 10
 processing epoch 11
 processing epoch 12
 processing epoch 13
 processing epoch 14
 processing epoch 15
 processing epoch 16
 processing epoch 17
 processing epoch 18
 processing epoch 19
 processing epoch 20
 processing epoch 21
 processing epoch 22
 processing epoch 23
 processing epoch 24
 processing epoch 25
 processing epoch 26
 processing epoch 27
 processing epoch 28
 processing epoch 29
 processing epoch 30
 processing epoch 31
 processing epoch 32
 processing epoch 33
 processing epoch 34
 processing epoch 35
 processing epoch 36
 processing epoch 37
 processing epoch 38
 processing epoch 39
 processing epoch 40
 processing epoch 41
 processing epoch 42
 processing epoch 43
 processing epoch 44
 processing epoch 45
 processing epoch 46
 processing epoch 47
 p

0.04668493451623727

In [6]:
userPred = []
itemPred = []
purchase = []
predictData = []
count = 0
average = 0
for i in ratingAll:
    average += i
average /= len(ratingAll)
print(average)
predictionsFile = open("predictions_Purchase.txt", 'w')
for l in open("pairs_Purchase.txt"):
    if l.startswith("reviewerID"):
        predictionsFile.write(l)
        continue
    u,i = l.strip().split('-')
    userPred.append(u)
    itemPred.append(i)
    purchase.append(0)
    predictData.append((u,i,0))
predDict = {'userID':userPred,
            'itemID':itemPred,
            'purchase':purchase}
predictDF = pd.DataFrame(predDict)
# predictData = Dataset.load_from_df(predictDF[['userID','itemID','purchase']],reader)
test_pred = algo.test(predictData)
print(len(predictData))

for pred in test_pred:
    u = pred[0]
    i = pred[1]
    if pred[3] >= 0.5:
         predictionsFile.write(u + '-' + i + ",1\n")
    else:
         predictionsFile.write(u + '-' + i + ",0\n")

4.23471
28000
